In [4]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

dataset_dir = '/home/huiwu1/dataset/coco-text/tfrecords/train-mini'
file_pattern = 'coco_%s_*.tfrecord'
split_name = 'train'
#num_samples=2
num_samples=3522
#num_samples=15124

file_pattern = os.path.join(dataset_dir, file_pattern % split_name)
slim = tf.contrib.slim

# Features in Pascal VOC TFRecords.
keys_to_features = {
    'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
    'image/format': tf.FixedLenFeature((), tf.string, default_value='jpeg'),
    'image/height': tf.FixedLenFeature([1], tf.int64),
    'image/width': tf.FixedLenFeature([1], tf.int64),
    'image/channels': tf.FixedLenFeature([1], tf.int64),
    'image/shape': tf.FixedLenFeature([3], tf.int64),
    'image/text/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
    'image/text/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
    'image/text/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
    'image/text/bbox/ymax': tf.VarLenFeature(dtype=tf.float32),
    'image/text/bbox/label': tf.VarLenFeature(dtype=tf.int64),
    'image/text/bbox/area': tf.VarLenFeature(dtype=tf.float32),
    }

items_to_handlers = {
    'image': slim.tfexample_decoder.Image('image/encoded', 'image/format'),
    'shape': slim.tfexample_decoder.Tensor('image/shape'),
    'text/bbox': slim.tfexample_decoder.BoundingBox(
    ['ymin', 'xmin', 'ymax', 'xmax'], 'image/text/bbox/'),
    'text/label': slim.tfexample_decoder.Tensor('image/text/bbox/label'),
    'text/area': slim.tfexample_decoder.Tensor('image/text/bbox/area'),
}
decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
labels_to_names = None
#if dataset_utils.has_labels(da)

ITEMS_TO_DESCRIPTIONS = {
    'image': 'A color image of varying height and width.',
    'shape': 'Shape of the image',
    'text/bbox': 'A list of bounding boxes, one per each object.',
    'text/label': 'A list of labels, one per each object.',
    'text/area': 'A list of areas',
}

#if reader is None:
reader = tf.TFRecordReader
dataset = slim.dataset.Dataset(
    data_sources=file_pattern,
    reader=reader,
    decoder=decoder,
    num_samples=num_samples,
    items_to_descriptions=ITEMS_TO_DESCRIPTIONS,
    num_classes=1,
    labels_to_names=labels_to_names)

provider = slim.dataset_data_provider.DatasetDataProvider(dataset)
[image, shape, glabels, gbboxes] = provider.get(['image',
                                                 'shape',
                                                 'text/label',
                                                 'text/bbox'])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    image1, shape1, glabels1, gbboxes1 = sess.run([image, shape, glabels, gbboxes])
    
    print("shape = {}".format(shape1))
    print("glabels = {}".format(glabels1))
    print("gbboxes = {}".format(gbboxes1))
    plt.imshow(image1)
    plt.show()
    coord.request_stop()
    coord.join(threads)

ValueError: Item [text/bbox/label] is invalid. Valid entries include: dict_keys(['image', 'shape', 'text/bbox', 'text/label', 'text/area', 'record_key'])